In [ ]:
# === colab configuration ===
# p.s. when training on colab, weights are saved on Drive (directory DSCAS/weights).
# p.p.s. skip this cell if running demo file locally!

! pip install torch-summary
! pip install ray
! pip install tensorboardX==2.5.1
! pip install tensorboard==2.11.2
from google.colab import drive, files
import sys

# setting paths
repository_path = '/content/deep-seasonal-color-analysis-system/'
dataset_path = repository_path + 'headsegmentation_dataset_ccncsa/'
dataset_path_drive = '/content/drive/MyDrive/DSCAS/headsegmentation_dataset_ccncsa/'
weights_path = repository_path + 'models/weights/'
weights_path_drive = '/content/drive/MyDrive/DSCAS/weights/'
checkpoint_path = repository_path + 'models/loss_selection/'
checkpoint_archive = '/content/' + checkpoint_path.split('/')[-2] + '.zip'

sys.path.insert(0, repository_path)

# cloning project repository and downloading dataset
drive.mount('/content/drive')
! test ! -d $repository_path && git clone https://github.com/mrcmich/deep-seasonal-color-analysis-system.git
! test ! -d $dataset_path && cp -R $dataset_path_drive $dataset_path
%cd $repository_path

# setting branch and pulling updates
branch = 'main'
! git checkout $branch
! git pull origin $branch

executing_on_colab = True

In [2]:
import os
import torch
from torch import nn, optim
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from models import dataset, training_and_testing
from models.cloud.UNet import unet
from metrics_and_losses import metrics
from utils import segmentation_labels, custom_transforms, model_names
from models import config
from slurm_scripts import slurm_config
from ray import tune
from ray.tune import CLIReporter
from functools import partial

try:
  executing_on_colab
except NameError:
  executing_on_colab = False
  weights_path = config.WEIGHTS_PATH
  dataset_path = config.DATASET_PATH
  checkpoint_path = config.LOSS_SELECTION_PATH

In [3]:
# defining transforms
image_transform = T.Compose([T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])
target_transform = T.Compose([T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO)])

# fetching dataset
n_classes = len(segmentation_labels.labels)
img_paths, label_paths = dataset.get_paths(dataset_path, file_name=config.DATASET_INDEX_NAME)
X_train, _, Y_train, _ = train_test_split(
    img_paths, label_paths, test_size=0.20, random_state=99, shuffle=True)
train_dataset = dataset.CcncsaDataset(X_train, Y_train, image_transform, target_transform)

# model parameters
model_name = "unet"
optimizer = optim.Adam

# setting up hyperparameters for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_epochs = 10
batch_size = 32
score_fn = metrics.batch_mIoU
learning_rate = 0.01
class_weights = torch.tensor(config.CLASS_WEIGHTS, device=device)

# Ray Tune parameters
cpus_per_trial = 0
gpus_per_trial = torch.cuda.device_count()
num_samples = 1  # Number of times each combination is sampled (n_epochs are done per sample)
metric = "val_loss"
metrics_columns = ["train_loss", "train_score", "val_loss", "val_score", "training_iteration"]
local_dir = checkpoint_path + model_names.MODEL_NAMES[model_name]
max_report_frequency = 600
reporter = CLIReporter(
    metric_columns=metrics_columns, max_report_frequency=max_report_frequency)

In [4]:
# === training with std. loss ===

checkpoint_dir = local_dir if executing_on_colab else (os.path.abspath('./' + local_dir) + '/')
cfg = {
    "lr": learning_rate,
    "lr_scheduler": "none",
    "batch_size": batch_size,
    "loss": "std",
    "from_checkpoint": False,
    "checkpoint_dir": checkpoint_dir
    }

model = unet.UNet(out_channels=n_classes)
loss_fn = nn.CrossEntropyLoss()

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=cfg,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

2023-02-03 07:33:42,511	INFO worker.py:1538 -- Started a local Ray instance.
2023-02-03 07:33:47,321	WARNING worker.py:1851 -- Warning: The actor ImplicitFunc is very large (32 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


== Status ==
Current time: 2023-02-03 07:33:48 (running for 00:00:02.19)
Memory usage on this node: 3.9/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.4 GiB heap, 0.0/3.7 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/loss_selection/UNet/train_model_2023-02-03_07-33-45
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-------------------+
| Trial name              | status   | loc               |
|-------------------------+----------+-------------------|
| train_model_178f9_00000 | RUNNING  | 172.28.0.12:68645 |
+-------------------------+----------+-------------------+




Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,train_score,training_iteration,trial_id,val_loss,val_score,warmup_time
train_model_178f9_00000,2023-02-03_08-19-07,True,,6beb311736ad452581ddfffcb501346f,0,195247e2f7e8,10,172.28.0.12,68645,2712.15,267.77,2712.15,1675412347,0,,0.146635,0.583151,10,178f9_00000,0.16159,0.582682,0.00314665


== Status ==
Current time: 2023-02-03 07:43:52 (running for 00:10:06.09)
Memory usage on this node: 6.9/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.4 GiB heap, 0.0/3.7 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/loss_selection/UNet/train_model_2023-02-03_07-33-45
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-------------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status   | loc               |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+----------+-------------------+--------------+---------------+------------+-------------+----------------------|
| train_model_178f9_00000 | RUNNING  | 172.28.0.12:68645 |     0.600656 |      0.351972 |   0.576567 |    0.356303 |                    2 |
+-------------------------+----------+-------------------+-

2023-02-03 08:19:07,517	INFO tune.py:762 -- Total run time: 2723.24 seconds (2721.11 seconds for the tuning loop).


== Status ==
Current time: 2023-02-03 08:19:07 (running for 00:45:21.12)
Memory usage on this node: 6.9/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.4 GiB heap, 0.0/3.7 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/loss_selection/UNet/train_model_2023-02-03_07-33-45
Number of trials: 1/1 (1 TERMINATED)
+-------------------------+------------+-------------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status     | loc               |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+------------+-------------------+--------------+---------------+------------+-------------+----------------------|
| train_model_178f9_00000 | TERMINATED | 172.28.0.12:68645 |     0.146635 |      0.583151 |    0.16159 |    0.582682 |                   10 |
+-------------------------+------------+----------

In [ ]:
# downloading updated training results if running on colab
! if test $executing_on_colab = 'True' ; then zip -r $checkpoint_archive $checkpoint_path ; fi
if executing_on_colab:
    files.download(checkpoint_archive)

In [9]:
# === training with std. loss ===

cfg["loss"] = "weighted"

model = unet.UNet(out_channels=n_classes)
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=cfg,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

2023-02-03 08:31:29,904	WARNING worker.py:1851 -- Warning: The actor ImplicitFunc is very large (32 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
2023-02-03 08:31:29,959	WARNING util.py:244 -- The `start_trial` operation took 0.799 s, which may be a performance bottleneck.


== Status ==
Current time: 2023-02-03 08:31:29 (running for 00:00:01.19)
Memory usage on this node: 4.1/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.4 GiB heap, 0.0/3.7 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/loss_selection/UNet/train_model_2023-02-03_08-31-28
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-------------------+
| Trial name              | status   | loc               |
|-------------------------+----------+-------------------|
| train_model_2796c_00000 | RUNNING  | 172.28.0.12:83571 |
+-------------------------+----------+-------------------+




Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,train_score,training_iteration,trial_id,val_loss,val_score,warmup_time
train_model_2796c_00000,2023-02-03_09-16-27,True,,7e6f181c218c41f2a75962bd8310949f,0,195247e2f7e8,10,172.28.0.12,83571,2693.82,268.739,2693.82,1675415787,0,,0.106541,0.588667,10,2796c_00000,0.114313,0.576191,0.0035944


== Status ==
Current time: 2023-02-03 08:41:34 (running for 00:10:05.83)
Memory usage on this node: 7.1/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.4 GiB heap, 0.0/3.7 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/loss_selection/UNet/train_model_2023-02-03_08-31-28
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-------------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status   | loc               |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+----------+-------------------+--------------+---------------+------------+-------------+----------------------|
| train_model_2796c_00000 | RUNNING  | 172.28.0.12:83571 |     0.401208 |      0.328908 |   0.390659 |      0.3283 |                    2 |
+-------------------------+----------+-------------------+-

2023-02-03 09:16:27,513	INFO tune.py:762 -- Total run time: 2699.33 seconds (2698.62 seconds for the tuning loop).


== Status ==
Current time: 2023-02-03 09:16:27 (running for 00:44:58.63)
Memory usage on this node: 7.1/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.4 GiB heap, 0.0/3.7 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/loss_selection/UNet/train_model_2023-02-03_08-31-28
Number of trials: 1/1 (1 TERMINATED)
+-------------------------+------------+-------------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status     | loc               |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+------------+-------------------+--------------+---------------+------------+-------------+----------------------|
| train_model_2796c_00000 | TERMINATED | 172.28.0.12:83571 |     0.106541 |      0.588667 |   0.114313 |    0.576191 |                   10 |
+-------------------------+------------+----------

In [ ]:
# downloading updated training results if running on colab
! if test $executing_on_colab = 'True' ; then zip -r $checkpoint_archive $checkpoint_path ; fi
if executing_on_colab:
    files.download(checkpoint_archive)